In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [2]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews-2.csv")
Hotel_City_Reviews=pd.read_csv("Hotel_Info_Country_City.csv", usecols=['Hotel_Name','City'])
Hotel_City_Reviews.head()

,Hotel_Name,City
0,Hotel Arena,Amsterdam
1,K K Hotel George,London
2,Apex Temple Court Hotel,London
3,The Park Grand London Paddington,London
4,Monhotel Lounge SPA,Paris


In [3]:
Vienna=Hotel_City_Reviews.loc[Hotel_City_Reviews['City']=='Vienna']
Vienna.Hotel_Name.unique()

array(['Hotel Bellevue Wien',
       'Hotel Am Konzerthaus Vienna MGallery by Sofitel',
       'Park Hyatt Vienna', 'Hilton Vienna',
       'Schlosshotel R mischer Kaiser', 'Hotel Mailberger Hof',
       'Novotel Wien City', 'The Levante Parliament A Design Hotel',
       'Vienna Sporthotel', 'Hotel Boltzmann',
       'Hotel ViennArt am Museumsquartier', 'Hotel Sans Souci Wien',
       'Palais Coburg Residenz', 'City Rooms', 'Mercure Vienna First',
       'Melia Vienna', 'Graben Hotel', 'Austria Trend Hotel Lassalle Wien',
       'The Guesthouse Vienna', 'Hotel Johann Strauss',
       'Hotel Mercure Wien Westbahnhof', 'Hotel Eitlj rg',
       'Mercure Wien Zentrum', 'Best Western Premier Kaiserhof Wien',
       'FourSide Hotel Suites Vienna', 'Mercure Secession Wien',
       'City Hotel Deutschmeister', 'FourSide Hotel Vienna City Center',
       'Hotel Vienna', 'Rainers Hotel Vienna',
       'Austria Trend Hotel Bosei Wien', 'Pertschy Palais Hotel',
       'Alma Boutique Hotel', 'Hilt

In [4]:
Hotel_Vienna = Hotel_Reviews[Hotel_Reviews['Hotel_Name'].isin(Vienna.Hotel_Name.unique())]
len(Hotel_Vienna)

39589

In [9]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LiverJoker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in Hotel_Vienna.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in Hotel_Vienna['Negative_Review'][i]) and "No Positive" in Hotel_Reviews['Positive_Review'][i]:
            continue
       elif "No Negative" in Hotel_Vienna['Negative_Review'][i]:
            reviewP = Hotel_Vienna['Positive_Review'][i]
       else:
            reviewN = Hotel_Vienna['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [word for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [word for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [26]:
len(corpusN)

28360

In [27]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [28]:
from functools import reduce

In [40]:
bowA = []
for line in corpusN:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

28360

In [41]:
wordSet=set(bowAnew)


In [42]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [43]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)


In [44]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PositiveReviews = PosRev.iloc[0:10, :]
PositiveReviews

,0
room,0.794871
hotel,0.559913
breakfast,0.490417
nothing,0.434028
staff,0.400348
small,0.398491
could,0.380076
bit,0.378447
rooms,0.374747
one,0.367632


In [45]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidf = {x:y for x,y in tfidf.items() if y!=0}
    tfidfBowA = np.append(tfidfBowA, tfidf)


#sum(tfidfBowA[1].values())
tfidfBowA[0]

{'alert': 0.8321929719979959,
 'close': 0.38492197874089357,
 'door': 0.6717340323334173,
 'guests': 0.38670894715448917,
 'hotel': 0.16236303024781132,
 'inside': 0.4434051429965363,
 'lift': 0.4451044278158229,
 'one': 0.24728285107793418,
 'properly': 0.4330724776808525,
 'sign': 0.5107278174924026}

In [46]:
OneHotList=[]
for i in range(0, len(tfidfBowA), 1):
    tempList=[]
    for key in tfidfBowA[i]:
        if tfidfBowA[i][key]>.05:
               tempList.append(key)
    OneHotList.append(tempList)

In [47]:
import collections 
import mlxtend

In [48]:
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori

In [49]:
oht = OnehotTransactions()
oht_ary = oht.fit(OneHotList).transform(OneHotList)  
df = pd.DataFrame(oht_ary, columns=oht.columns_)  
df

,aa,ab,aback,abandoned,abbondant,abby,abd,ability,abit,able,...,zimmerman,zink,zipped,zippererstra,zobudilo,zone,zoo,zoom,zur,zzzound
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.050564,[bathroom],1
1,0.057440,[bed],1
2,0.069076,[bit],1
3,0.121544,[breakfast],1
4,0.067595,[could],1
5,0.148025,[hotel],1
6,0.097743,[nothing],1
7,0.058745,[one],1
8,0.250071,[room],1
9,0.065127,[rooms],1
